# Importing Libraries

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import timm
import torchvision.models as models
import torchvision.transforms as transforms
import cv2
from PIL import Image
from skimage.feature import hog, local_binary_pattern

c:\Users\Pdogg Windows10\Desktop\Semester 7\IT499 - Major Project II\Project\majorproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# HFF CNN Models

In [ ]:
# Define image size
image_size = (224, 224)  # Suitable for ResNet, DenseNet, and similar architectures

# Define supported models
cnn_models = {
    'ResNet-50': models.resnet50(pretrained=True),
    'SE-ResNet-50': timm.create_model('seresnet50', pretrained=True),
    'ResNeXt-50': timm.create_model('resnext50_32x4d', pretrained=True),
    'ResNeSt-50': timm.create_model('resnest50d', pretrained=True),
}

c:\Users\Pdogg Windows10\Desktop\Semester 7\IT499 - Major Project II\Project\majorproject\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Pdogg Windows10\Desktop\Semester 7\IT499 - Major Project II\Project\majorproject\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Pdogg Windows10\Desktop\Semester 7\IT499 - Major Project II\Project\majorproject\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 a

In [9]:
train_dir = 'Data224/train'
test_dir = 'Data224/test'

In [3]:
# Image Transformations (Same for all CNN models)
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Function to load the selected model and remove the last layer
def load_cnn_model(model_name):
    if model_name not in cnn_models:
        raise ValueError(f"Model '{model_name}' not found in supported models!")

    model = cnn_models[model_name]
    model = nn.Sequential(*list(model.children())[:-1])  # Remove last FC layer
    model.eval()  # Set to evaluation mode
    return model

In [5]:
# Function to extract CNN features
def extract_cnn_features(image, model):
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model(image)
    return features.flatten().numpy()

In [6]:
# Function to extract handcrafted features (HOG, LBP, Color Histogram)
def extract_handcrafted_features(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # HOG Feature Extraction
    hog_features = hog(image_gray, pixels_per_cell=(16, 16), cells_per_block=(1, 1), feature_vector=True)

    # LBP Feature Extraction
    lbp = local_binary_pattern(image_gray, P=24, R=3, method="uniform")
    lbp_hist, _ = np.histogram(lbp, bins=np.arange(0, 27), density=True)

    # Color Histogram (RGB)
    hist_r = cv2.calcHist([image], [0], None, [256], [0, 256]).flatten()
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256]).flatten()
    hist_b = cv2.calcHist([image], [2], None, [256], [0, 256]).flatten()

    # Normalize and concatenate features
    hist_rgb = np.concatenate([hist_r, hist_g, hist_b]) / np.linalg.norm(hist_r)
    
    return np.concatenate([hog_features, lbp_hist, hist_rgb])

In [ ]:
# Function to process dataset (train or test) using a specific CNN model
def process_dataset(dataset_dir, dataset_type, model_name):
    # Load the selected model
    model = load_cnn_model(model_name)

    features = []
    labels = []
    class_mapping = {}
    class_id = 0

    # Define save directory
    save_dir = os.path.join("features", model_name, dataset_type)
    os.makedirs(save_dir, exist_ok=True)  # Ensure directory exists

    for class_folder in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_folder)
        if not os.path.isdir(class_path):
            continue  # Skip non-directory files

        if class_folder not in class_mapping:
            class_mapping[class_folder] = class_id
            class_id += 1

        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            image = cv2.imread(img_path)

            if image is None:
                print(f"Skipping invalid image: {img_path}")
                continue
            
            # Extract CNN and Handcrafted Features
            resnet_feat = extract_cnn_features(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)), model)
            handcrafted_feat = extract_handcrafted_features(image)
            
            # Combine both feature sets
            combined_features = np.concatenate([resnet_feat, handcrafted_feat])

            features.append(combined_features)
            labels.append(class_mapping[class_folder])

     # Convert to NumPy arrays and save
    features = np.array(features)
    labels = np.array(labels)
    
    np.save(os.path.join(save_dir, f"hff_features_{dataset_type}.npy"), features)
    np.save(os.path.join(save_dir, f"hff_labels_{dataset_type}.npy"), labels)
    
    print(f"✅ {dataset_type.capitalize()} Feature Extraction Completed for {model_name}! Extracted {features.shape[1]} features per image.")

## DenseNet

In [10]:
process_dataset(train_dir, "train", "DenseNet-121")
process_dataset(test_dir, "test", "DenseNet-121")

✅ Train Feature Extraction Completed for DenseNet-121! Extracted 52734 features per image.
✅ Test Feature Extraction Completed for DenseNet-121! Extracted 52734 features per image.


## SE-ResNet-50

In [11]:
process_dataset(train_dir, "train", "SE-ResNet-50")
process_dataset(test_dir, "test", "SE-ResNet-50")

✅ Train Feature Extraction Completed for SE-ResNet-50! Extracted 4606 features per image.
✅ Test Feature Extraction Completed for SE-ResNet-50! Extracted 4606 features per image.


## ResNext-50

In [12]:
process_dataset(train_dir, "train", "ResNeXt-50")
process_dataset(test_dir, "test", "ResNeXt-50")

✅ Train Feature Extraction Completed for ResNeXt-50! Extracted 4606 features per image.
✅ Test Feature Extraction Completed for ResNeXt-50! Extracted 4606 features per image.


## ResNeSt-50

In [13]:
process_dataset(train_dir, "train", "ResNeSt-50")
process_dataset(test_dir, "test", "ResNeSt-50")

✅ Train Feature Extraction Completed for ResNeSt-50! Extracted 4606 features per image.
✅ Test Feature Extraction Completed for ResNeSt-50! Extracted 4606 features per image.
